In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
try:
    print("Repo path:", repo_path)
except:
    os.chdir("../")
    repo_path = os.getcwd()
    print("Repo path:", repo_path)

Repo path: /Users/tx44ru/Projects/Kaggle/DisasterTweets


In [131]:
data_path = "data"
use_dir = "use"
glove_dir = "glove"

os.chdir(repo_path)

In [132]:
os.listdir(data_path)

['test.csv', 'train.csv', 'sample_submission.csv']

In [235]:
df = pd.read_csv(os.path.join(data_path, "train.csv"))
# df["text_token"] = df["text"].apply(lambda x: [token.lower() for token in x.split()])
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Text data cleanning

In [236]:
word = "shapent"
# word  = "".join([char.lower() if char not in string.punctuation else " " for char in word ]).split()
# word
"'"+word

"'shapent"

In [237]:

import nltk
import re

ps = nltk.PorterStemmer()

def txt_cleanning(text, tokenize=True, stem=False):
    """ cleanning:
            lowercase
            remove numbers
            remove punctuation
            stem words
    """
    if not isinstance(text, list):
        # workaround in order to be able to re-run it
        text  = "".join([char.lower() for char in text ])
        text = re.sub('[0-9]+', '', text)
        text = text.split()
        out_text = []
        for word in text:
            if "\\x" in word:
                continue
            if "http" in word:
                out_text.append("http")
                continue
            if word == "ain't":
                out_text += ["is", "not"]
                continue
            if word == "won't":
                out_text += ["will", "not"]
                continue
            if word and word[0] in string.punctuation:
                word = word[1:]
            if word and word[-1] in string.punctuation:
                word = word[:-1]
            if len(word) > 4 and "n't" in word[-3:]:
                out_text.append(word[:-3])
                out_text.append("not")
            elif "'" in word:
                new_words = word.split("'")
                out_text += [new_words[0], "'" + new_words[1]]
            elif "-" in word:
                out_text += word.split("-")
            elif word:
                word = re.sub("[^a-z]+", "", word)
                out_text  += "".join([char if char.isalpha()
                                              else " " for char in word ]).split()
    out_text = [word.strip() for word in out_text if word.strip()]
    if stem:
        text = [ps.stem(word) for word in out_text]  # remove stopwords and stemming
    if not tokenize:
        out_text = " ".join([str(elem) for elem in out_text])
    return out_text

df['text_token'] = df['text'].apply(lambda x: txt_cleanning(x))
df.head(10)

,id,keyword,location,text,target,text_token
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala..."
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,"[rockyfire, update, california, hwy, closed, i..."
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,"[flood, disaster, heavy, rain, causes, flash, ..."
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,"[i, 'm, on, top, of, the, hill, and, i, can, s..."
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,"[there, 's, an, emergency, evacuation, happeni..."
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,"[i, 'm, afraid, that, the, tornado, is, coming..."


In [204]:
def clean_hashtag(inpt, returnString=False):
    if not isinstance(inpt, list):
        inpt = inpt.split()
    inpt = [token.strip() for token in inpt]
    ret = [token[1:] if len(token) > 1 and token[0] == "#" else token for token in inpt]
    if returnString:
        return " ".join([token for token in ret])
    return ret

In [164]:
# df["text_token"] = df["text_token"].apply(clean_hashtag)
# df.head()

### Train/Test Split

In [222]:
from sklearn.model_selection import train_test_split

In [223]:
# X_df, y_df = df["text"], df["target"]
X_df, y_df = df["text_token"], df["target"]

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, random_state=0)

In [225]:
print("train:", X_train.shape)
print("test:", X_test.shape)

train: (5709,)
test: (1904,)


In [226]:
X_train.head()

5244    [refugio, oil, spill, may, have, been, costlie...
4860    [julian, knight, scvsupremecourt, dismisses, m...
6538    [electricity, cant, stop, scofield, nigga, sur...
5175    [meek, mill, begging, nicki, minaj, to, let, h...
5820    [chinas, stock, market, crash, this, summer, h...
Name: text_token, dtype: object

### USE

In [11]:
import tensorflow_hub as hub
import tensorflow as tf
from tqdm import tqdm

try:
    print("USE module:\n", use_module)
except:
    use_module = hub.load(use_dir)
    print("USE module:\n", use_module)

USE module:
 <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fd439ec6550>


In [12]:
def get_use(text, use_module):
    text = [text]
    emb = use_module.signatures["response_encoder"](
        input=tf.constant(text),
        context=tf.constant(text))["outputs"].numpy()
    return emb

In [13]:
test = "This is a test"
get_use(test, use_module).squeeze().shape

(512,)

In [733]:
X_train_use = []

for text in tqdm(X_train):
    X_train_use.append(get_use(text, use_module).squeeze())

100%|██████████| 5709/5709 [03:14<00:00, 29.40it/s]


In [745]:
X_test_use = []

for text in tqdm(X_test):
    X_test_use.append(get_use(text, use_module).squeeze())

100%|██████████| 1904/1904 [01:04<00:00, 29.55it/s]


In [742]:
X_train_use[0].shape

(512,)

In [ ]:
X_train_use = []

for text in tqdm(X_train):
    X_train_use.append(get_use(text, use_module).squeeze())

### TF-IDF

In [34]:
# Create the vectorizer method
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df=3, max_df=0.8, stop_words="english")

In [35]:
tfidf_vec.fit(X_train)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
X_train_tfidf = tfidf_vec.transform(X_train).toarray()
X_test_tfidf = tfidf_vec.transform(X_test).toarray()

In [36]:
X_train_tfidf.shape

(5709, 3372)

## GloVe

In [186]:
%%time
glove_voc = []
glove_weights = []
glove_dimensions = 200
glove_file = os.path.join(glove_dir, "glove.twitter.27B.{}d.txt".format(glove_dimensions))
with open(glove_file, 'r') as f:
    for line in f:
        values = line.split()
        if len(values[1:]) == glove_dimensions:
            glove_voc.append(values[0])
            glove_weights.append(values[1:])
print("GloVe embeddings imported.\nVector size: ")
print(len(glove_weights[-1]))
print()

GloVe embeddings imported.
Vector size: 
200

CPU times: user 31.8 s, sys: 8.74 s, total: 40.5 s
Wall time: 41.8 s


In [187]:
glove_weights = np.asarray(glove_weights, "float32")

In [188]:
(glove_voc_length, glove_weights_length) = glove_weights.shape

## Naive Bayes classifier

In [746]:
X_train_input = X_train_use
X_test_input = X_test_use

In [744]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_input, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [747]:
# Print out the model's accuracy
nb.score(X_test_input, y_test)

0.7478991596638656

## Deep Net classifier

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(1)

In [778]:
class SimpleNet(nn.Module):
    
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, round(input_size / 2))
        self.fc2 = nn.Linear(round(input_size / 2), 100)
        self.fc3 = nn.Linear(100, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
    
    def predict(self, x):
        #Apply sigmoid to output.
        pred = self.forward(x)
        return pred.round()

In [930]:
#Deeper NN
class SimpleNet_v2(nn.Module):
    
    def __init__(self, input_size):
        super(SimpleNet_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout1 = nn.Dropout(p = 0.6)
        self.fc2 = nn.Linear(512, 512)
        self.dropout2 = nn.Dropout(p = 0.3)
        self.fc3 = nn.Linear(512, 32)
        self.fc4 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = self.dropout1(F.relu(self.fc1(x)))
        x = self.dropout2(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        out = torch.sigmoid(self.fc4(x))
        return out
    
    def predict(self, x):
        #Apply sigmoid to output.
        pred = self.forward(x)
        return pred.round()


In [931]:
X_train_input = X_train_use
X_test_input = X_test_use

In [932]:
def get_dimension(inp):
    if isinstance(inp, list):
        return len(inp[0])
    if isinstance(inp, np.ndarray):
        return inp.shape[1]
    return -1

In [933]:
simple_classifier = SimpleNet_v2(get_dimension(X_train_input))

In [934]:
def save_model(epoch, model, optimizer, scheduler, name):
    train_state = {    
    'model' : model,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'scheduler': scheduler.state_dict()
    }
    torch.save(train_state, name)
    pass

In [935]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(simple_classifier.parameters(), lr=0.001, weight_decay = 0.007);
lr_scheduler = ReduceLROnPlateau(optimizer, 'max', factor = 0.1, patience = 5, verbose=True)

# train_loader, test_loader = create_dataloaders(balanced_train_images, balanced_train_labels, pr_test_img, pr_test_labels)
best_accuracy = 0.0

### Training

In [936]:
%%time
EARLY_STOP = 0
EPOCH = 50
batchsize = 128
model_name = 'SimpleNet_use_3layers.pt'

for epoch in range(EPOCH):  # loop over the dataset multiple times
    if EARLY_STOP >= 21:
        print("STOPPED EARLY!!")
        break
    simple_classifier.train()
    running_loss = 0.0
    for i in range(0, len(y_train), batchsize):
        
        inputs = torch.tensor(X_train_input[i:i+batchsize]).float()
        lbs = torch.tensor(y_train[i:i+batchsize].values).float()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = simple_classifier(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i/batchsize) % 30 == 29:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, (i/batchsize) + 1, running_loss / 1000))
            running_loss = 0.0

    simple_classifier.eval()
    correct = 0
    total = 0
    for j in range(0, len(y_test), batchsize):
        labels = torch.tensor(y_test[j:j+batchsize].values)
        outputs = simple_classifier.predict(torch.tensor(X_test_input[j:j+batchsize]).float())
        total += labels.shape[0]
        correct += outputs.squeeze().eq(labels).sum().item()
    cur_accuracy = correct / total
    print('Accuracy on EPOCH %d test images: %d %%' % (epoch+1, 100 * cur_accuracy))   
    lr_scheduler.step(cur_accuracy)
    if cur_accuracy > best_accuracy:
            best_accuracy = cur_accuracy
            print("Saving current model!!!")
            print("Detailed Accuracy: %f" %(best_accuracy))
            save_model(epoch, simple_classifier, optimizer, lr_scheduler, model_name)
            EARLY_STOP = 0
    EARLY_STOP += 1
print('Finished Training')

[1,    30] loss: 0.021
Accuracy on EPOCH 1 test images: 58 %
Saving current model!!!
Detailed Accuracy: 0.581408
[2,    30] loss: 0.019
Accuracy on EPOCH 2 test images: 81 %
Saving current model!!!
Detailed Accuracy: 0.815126
[3,    30] loss: 0.015
Accuracy on EPOCH 3 test images: 80 %
[4,    30] loss: 0.013
Accuracy on EPOCH 4 test images: 81 %
[5,    30] loss: 0.013
Accuracy on EPOCH 5 test images: 81 %
Saving current model!!!
Detailed Accuracy: 0.819328
[6,    30] loss: 0.013
Accuracy on EPOCH 6 test images: 82 %
Saving current model!!!
Detailed Accuracy: 0.820903
[7,    30] loss: 0.013
Accuracy on EPOCH 7 test images: 81 %
[8,    30] loss: 0.013
Accuracy on EPOCH 8 test images: 81 %
[9,    30] loss: 0.012
Accuracy on EPOCH 9 test images: 81 %
[10,    30] loss: 0.013
Accuracy on EPOCH 10 test images: 82 %
[11,    30] loss: 0.013
Accuracy on EPOCH 11 test images: 81 %
[12,    30] loss: 0.013
Accuracy on EPOCH 12 test images: 82 %
Epoch    12: reducing learning rate of group 0 to 1.00

## Encode words

### Vocabulary to integer

In [238]:
import string

class Vocabulary():
    def __init__(self):
        self.PAD = 0
        self.index2word = {self.PAD: "PAD"}
        self.word2index = {"PAD": self.PAD}
        self.size = 1
        self.oov = {}
        
    
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.size
            self.index2word[self.size] = word 
            self.size += 1
        return self.size-1
    
    def add_list(self, voc_list):
        if not isinstance(voc_list, list):
            voc_list = voc_list.split()
        for word in voc_list:
            self.add_word(word)
        return "1"

    def get_word(self, index):
        try:
            return self.index2word[index]
        except:
            return "Error: INDEX TOO BIG"

    def get_index(self, word):
        try:
            return self.word2index[word]
        except:
            if word not in self.oov:
                self.oov[word] = 1
            else:
                self.oov[word] += 1
            return -1
    def res_oov(self):
        self.oov = {}
        return "1"

In [239]:
my_vocabulary = Vocabulary()
my_vocabulary.add_list(glove_voc)

'1'

In [240]:
my_vocabulary.get_index("?")

15

In [19]:
glove_voc[553]

'try'

## LSTM

In [429]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(10)

In [430]:
embeddings = nn.Embedding(glove_voc_length+1, glove_weights_length, padding_idx=0)
embeddings.weight = nn.Parameter(torch.cat([torch.zeros(1, glove_dimensions), torch.tensor(glove_weights)], dim=0))
embeddings.weight.requires_grad = False

In [431]:
inpt = torch.LongTensor([[glove_voc_length]])
embeddings(inpt) == torch.tensor(glove_weights[glove_voc_length-1])

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True, True,
          Tr

In [521]:
class LSTMClassifier(nn.Module):
    def __init__(self, hidden_size, vocabulary_length, emb_dim, emb_weights, 
                 lstm_layers=1, output_size=1, oov_id=-1):
        super(LSTMClassifier, self).__init__()
        self.oov_id = oov_id
        self.embedding_dim = emb_dim
        self.embeddings = nn.Embedding(vocabulary_length, emb_dim, padding_idx=0)
        self.embeddings.weight = emb_weights
        self.embeddings.weight.requires_grad = False
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.layers = lstm_layers
        self.lstm = nn.LSTM(self.embedding_dim, hidden_size, lstm_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.dropout1 = nn.Dropout(p = 0.5)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.embeddings.weight.requires_grad = False
        
    def forward(self, sentence_arr):
        batch_size = sentence_arr.shape[0]
        h0 = Variable(torch.zeros(self.layers, batch_size, self.hidden_size)).requires_grad_() 
        c0 = Variable(torch.zeros(self.layers, batch_size, self.hidden_size)).requires_grad_()
        inpt = self.create_input(sentence_arr)
        out, (out_state, final_cell_state) = self.lstm(inpt, (h0.detach(), c0.detach()))
        x = F.relu(out[:, -1, :])
        x = self.dropout1(F.relu(self.fc1(x)))
        out = torch.sigmoid(self.fc2(x))
        return out, (out_state, final_cell_state)
    
    def predict(self, sentence_arr):
        pred, _ = self.forward(sentence_arr)
        return pred.round()
        
    def create_input(self, sentence_arr):
        """
        code inspired from https://stackoverflow.com/questions/53316174/
        using-pre-trained-word-embeddings-how-to-create-vector-for-unknown-oov-token
        """
        sentence_arr = sentence_arr
        mask = (sentence_arr==self.oov_id).long()
#         print(mask.sum())
        embed_known = self.embeddings((1-mask)*sentence_arr)
        embed_random = mask.unsqueeze(-1)*torch.nn.Parameter(data=torch.rand(embed_known.shape))
        emb = embed_known + embed_random
        emb = emb.detach()
        return emb

In [522]:
batch_size = 80
emb_weigths = nn.Parameter(torch.cat([torch.zeros(1, glove_dimensions), torch.tensor(glove_weights)], dim=0))
lstm_cf = LSTMClassifier(200, glove_voc_length+1, glove_weights_length,  emb_weigths, lstm_layers=1)

In [523]:
# lstm_cf.create_input(torch.tensor(data=[-1,-1]))

### DataLoaders

In [524]:
def sentence_to_index(lst, sentence_length, vocabulary):
    ret = [vocabulary.get_index(word) for word in lst]
    if len(ret) < sentence_length:
        for i in range(sentence_length - len(ret)):
            ret.append(0)
    else:
        ret = ret[:sentence_length]
    return ret

In [525]:
def create_dataloader(X, y, vocabulary, batch_size=64, sentence_length=15):
    X = np.vstack(X.apply(lambda x: sentence_to_index(x, sentence_length, vocabulary)))
    dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(y.values))
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    return dataloader

In [526]:
my_vocabulary.res_oov()
def create_numpy(X, y, vocabulary, batch_size=64, sentence_length=15):
    X = np.vstack(X.apply(lambda x: sentence_to_index(x, sentence_length, vocabulary)))
    return X, y.values

X_train_lstm, y_train_lstm = create_numpy(X_train, y_train, my_vocabulary, batch_size=batch_size)
X_test_lstm, y_test_lstm = create_numpy(X_test, y_test, my_vocabulary, batch_size=batch_size)

In [527]:
df["sentence_length"] = df["text_token"].apply(lambda x: len(x))
df["sentence_length"].describe()

count    7613.000000
mean       14.697360
std         5.817742
min         1.000000
25%        10.000000
50%        15.000000
75%        19.000000
max        32.000000
Name: sentence_length, dtype: float64

In [528]:
train_dataloader = create_dataloader(X_train, y_train, my_vocabulary, batch_size=batch_size)
test_dataloader = create_dataloader(X_test, y_test, my_vocabulary, batch_size=batch_size)

In [529]:
x = my_vocabulary.oov
x = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

total = 0
for i,j in x.items():
    total += j
print(total)
x

14360


{'bioterror': 66,
 'prebreak': 60,
 'soudelor': 52,
 'bioterrorism': 48,
 'bestnaijamade': 48,
 'disea': 38,
 'funtenna': 34,
 'crematoria': 30,
 'udhampur': 26,
 'spos': 18,
 'inundation': 18,
 'irandeal': 18,
 'mediterran': 16,
 'icemoon': 14,
 'djicemoon': 14,
 'microlight': 14,
 'linkury': 14,
 'runion': 14,
 'mhtw': 14,
 'fnet': 14,
 'animalrescue': 14,
 'canaanites': 14,
 'dorret': 12,
 'wheavenly': 12,
 'prophetmuhammad': 12,
 'sinjar': 12,
 'mikeparractor': 12,
 "'all": 12,
 'eblokeve': 12,
 '&gt': 12,
 'youngheroesid': 10,
 'pantherattack': 10,
 'twia': 10,
 'viralspell': 10,
 'naved': 10,
 'sittwe': 10,
 'strategicpatience': 10,
 'usagov': 10,
 'summerfate': 10,
 'injuryi': 10,
 'gtgtgt': 10,
 'kerricktrial': 10,
 'socialnews': 10,
 'nasahurricane': 10,
 'rockyfire': 8,
 'wisenews': 8,
 'waimate': 8,
 'unsuckdcmetro': 8,
 'carryi': 8,
 'kurtschlichter': 8,
 'beforeitsnews': 8,
 'votejkt': 8,
 'raynbowaffair': 8,
 'diamondkesawn': 8,
 'ramag': 8,
 'warfighting': 8,
 'lulgzimbe

### Utils

In [530]:
def save_model(epoch, model, optimizer, scheduler, name):
    train_state = {    
    'model' : model,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'scheduler': scheduler.state_dict()
    }
    torch.save(train_state, name)
    pass

In [533]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm_cf.parameters(), lr=0.01, weight_decay = 0.015);
# optimizer = torch.optim.SGD(lstm_cf.parameters(), lr=0.001, weight_decay = 0.005) 
lr_scheduler = ReduceLROnPlateau(optimizer, 'max', factor = 0.1, patience = 10, verbose=True)

best_accuracy = 0.0

### Training

In [534]:
%%time
EARLY_STOP = 0
EPOCH = 60
model_name = 'lstm.pt'

for epoch in range(EPOCH):
#     if EARLY_STOP >= 21:
#         print("STOPPED EARLY!!")
#         break
    lstm_cf.train()
    running_loss = 0.0
    for i, (batch_X, batch_y) in enumerate(train_dataloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs, _ = lstm_cf(batch_X)
        loss = criterion(outputs, batch_y.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

    lstm_cf.eval()
    correct = 0
    total = 0
    for test_X, test_y in test_dataloader:
        outputs = lstm_cf.predict(test_X)
        total += test_y.shape[0]
        correct += outputs.squeeze().eq(test_y).sum().item()
    cur_accuracy = correct / total
    print('Accuracy on EPOCH %d test images: %d %%' % (epoch+1, 100 * cur_accuracy))
    lr_scheduler.step(cur_accuracy)
    if cur_accuracy > best_accuracy:
            best_accuracy = cur_accuracy
            print("Saving current model!!!")
            print("Detailed Accuracy: %f" %(best_accuracy))
            save_model(epoch, lstm_cf, optimizer, lr_scheduler, model_name)
            EARLY_STOP = 0
    EARLY_STOP += 1
print('Finished Training')

[1,    30] loss: 0.018
[1,    60] loss: 0.016
Accuracy on EPOCH 1 test images: 78 %
Saving current model!!!
Detailed Accuracy: 0.782563
[2,    30] loss: 0.015
[2,    60] loss: 0.014
Accuracy on EPOCH 2 test images: 79 %
Saving current model!!!
Detailed Accuracy: 0.792542
[3,    30] loss: 0.014
[3,    60] loss: 0.015
Accuracy on EPOCH 3 test images: 80 %
Saving current model!!!
Detailed Accuracy: 0.800420
[4,    30] loss: 0.015
[4,    60] loss: 0.014
Accuracy on EPOCH 4 test images: 78 %
[5,    30] loss: 0.014
[5,    60] loss: 0.014
Accuracy on EPOCH 5 test images: 79 %
[6,    30] loss: 0.014
[6,    60] loss: 0.015
Accuracy on EPOCH 6 test images: 76 %
[7,    30] loss: 0.015
[7,    60] loss: 0.014
Accuracy on EPOCH 7 test images: 80 %
Saving current model!!!
Detailed Accuracy: 0.804622
[8,    30] loss: 0.014
[8,    60] loss: 0.015
Accuracy on EPOCH 8 test images: 74 %
[9,    30] loss: 0.015
[9,    60] loss: 0.014
Accuracy on EPOCH 9 test images: 79 %
[10,    30] loss: 0.014
[10,    60] 

### Load best model

In [535]:
# give best_classifier the class of the model you want to load
best_classifier = lstm_cf

best_classifier.load_state_dict(torch.load('lstm.pt')['state_dict'])

<All keys matched successfully>

In [536]:
X_test_input = torch.tensor(X_test_lstm).long()
X_train_input = torch.tensor(X_train_lstm).long()

In [537]:
best_classifier.eval()
preds = best_classifier.predict(X_test_input).detach().numpy()
print("Accuracy - Test:")
print((preds.squeeze() == y_test).sum().item() / y_test.shape[0])

Accuracy - Test:
0.8146008403361344


In [538]:
best_classifier.eval()
preds = best_classifier.predict(X_train_input).detach().numpy()
print("Accuracy - Train:")
print((preds.squeeze() == y_train).sum().item() / y_train.shape[0])

Accuracy - Train:
0.837099316868103


### Submit

In [350]:
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))
# df_test["text"] = df_test["text"].apply(lambda x: txt_cleanning(x))
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [351]:
submit_X = df_test["text"].apply(txt_cleanning)

In [352]:
submit_X_tfidf = tfidf_vec.transform(submit_X).toarray()

NameError: name 'tfidf_vec' is not defined

In [943]:
submit_X_use = []

for text in tqdm(submit_X):
    submit_X_use.append(get_use(text, use_module).squeeze())

100%|██████████| 3263/3263 [01:52<00:00, 29.04it/s]


In [353]:
submit_X_inputs, _ = create_numpy(submit_X, submit_X, my_vocabulary)

In [354]:
submit_preds = best_classifier.predict(torch.tensor(submit_X_inputs).long()).detach().numpy().astype(int)

In [355]:
to_submit_df = pd.DataFrame(submit_preds, columns=["target"])

to_submit_df = pd.concat((df_test["id"], to_submit_df), axis = 1)

to_submit_df.to_csv("submission.csv", index=False)

In [356]:
!kaggle competitions submit -c nlp-getting-started  -f submission.csv -m "Test"

100%|██████████████████████████████████████| 22.2k/22.2k [00:05<00:00, 4.39kB/s]
Successfully submitted to Real or Not? NLP with Disaster Tweets